In [ ]:
import numpy as np
import tvm
from tvm import te

In [ ]:
def padding(X, ph, pw, val=0):
    assert len(X.shape) >= 2
    nh,nw = X.shape[-2],X.shape[-1]
    return te.compute(
        (*X.shape[0:-2],nh+ph*2,nw+pw*2),
        lambda *i:te.if_then_else(
            te.any(i[-2]<ph, i[-2]>=nh+ph, i[-1]<pw, i[-1]>=nw+pw),
            val,X[i[:-2]+(i[-2]-ph,i[-1]-pw)]),name='PaddedX')
    

In [ ]:
A=te.placeholder((2,3,4))
B=padding(A,1,2)
s=te.create_schedule(B.op)
mod = tvm.build(s,[A,B])

a=tvm.nd.array(np.ones((2,3,4),dtype='float32'))
b=tvm.nd.array(np.empty((2,5,8),dtype='float32'))
mod(a,b)
print(b)

### Convolution

In [ ]:
def conv_out_size(n,k,p,s):
    return (n-k+2*p)//s + 1

In [ ]:
def conv(oc,ic,nh,nw,kh,kw,ph=0,pw=0,sh=0,sw=1):
    """Convolution
    oc, ic : output and input channels
    nh, nw : input width and height
    kh, kw : kernel width and height
    ph, pw : height and width padding sizes, default 0
    sh, sw : height and width strides, default 1
    """
    ric=te.reduce_axis((0,ic),name='ric')
    rkh=te.reduce_axis((0,kh),name='rkh')
    rkw=te.reduce_axis((0,kw),name='rkw')
    
    oh = conv_out_size(nh,kh,ph,sh)
    ow = conv_out_size(nw,kw,pw,sw)
    
    X=te.placeholder((ic,nh,nw),name='X')
    K=te.placeholder((oc,ic,kh,kw),name='K')
    paddedX = padding(X,ph,pw) if ph *pw !=0 else X
    Y = te.compute(
        (oc,oh,ow),
        lambda c,i,j:te.sum(
            paddedX[ric,i*sh+rkh,j*sw+rkw] * K[c,ric,rkh,rkw],
            axis=[ric,rkh,rkw]),name='Y')
    return X,K,Y,paddedX

In [ ]:
def get_conv_data(oc,ic,n,k,p=0,s=1,constructor=None):
    np.random.seed(0)
    data = np.random.normal(size=(ic,n,n)).astype('float32')
    weight = np.random.normal(size=(oc,ic,k,k)).astype('float32')
    on = conv_out_size(n,k,p,s)
    out = np.empty((oc,on,on), dtype='float32')
    if constructor:
        data, weight, out = (constructor(x) for x in [data, weight, out])
    return data,weight,out

In [ ]:
oc,ic,n,k,p,s = 4,6,12,3,1,1
X,K,Y,_ = conv(oc,ic,n,n,k,k,p,p,s,s)
sch = te.create_schedule(Y.op)
mod = tvm.build(sch,[X,K,Y])
# print(tvm.lower(sch,[X,K,Y], simple_mode=True))

data, weight, out = get_conv_data(oc, ic, n, k, p, s, tvm.nd.array)
mod(data, weight, out)

In [ ]:
import mxnet as mx

def get_conv_data_mxnet(oc, ic, n, k, p, s, ctx='cpu'):
    ctx = getattr(mx, ctx)()
    data, weight, out = get_conv_data(oc, ic, n, k, p, s,
                                      lambda x: mx.nd.array(x, ctx=ctx))
    data, out = data.expand_dims(axis=0), out.expand_dims(axis=0)
    bias = mx.nd.zeros(out.shape[1], ctx=ctx)
    return data, weight, bias, out

def conv_mxnet(data, weight, bias, out, k, p, s):
    mx.nd.Convolution(data, weight, bias, kernel=(k,k), stride=(s,s),
                      pad=(p,p), num_filter=out.shape[1], out=out)

data, weight, bias, out_mx = get_conv_data_mxnet(oc, ic, n, k, p, s)
conv_mxnet(data, weight, bias, out_mx, k, p, s)

In [ ]:
np.testing.assert_allclose(out_mx[0].asnumpy(), out.asnumpy(), atol=1e-5)